In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import ipywidgets as widgets
from IPython.display import display

# === Daten einlesen ===
weather_df = pd.read_csv("weather_europe_2020.csv")  # enthält: City, Date, Temperature
energy_df = pd.read_csv("energiepreise_europe.csv", low_memory=False)

# === Wetterdaten vorbereiten ===
weather_df["Date"] = pd.to_datetime(weather_df["Date"])

# === Energiedaten vorbereiten ===
energy_df["hora"] = energy_df["hora"].astype(str)
energy_df["hora"] = energy_df["hora"].str.zfill(2) + ":00"

# 24:00 Fälle korrigieren
mask_24 = energy_df["hora"] == "24:00"
energy_df.loc[mask_24, "hora"] = "00:00"
energy_df.loc[mask_24, "fecha"] = pd.to_datetime(energy_df.loc[mask_24, "fecha"]) + pd.Timedelta(days=1)

# Kombiniertes datetime-Feld
energy_df["fecha"] = pd.to_datetime(energy_df["fecha"])
energy_df["datetime"] = pd.to_datetime(energy_df["fecha"].dt.strftime("%Y-%m-%d") + " " + energy_df["hora"])

# === Mapping: Land → Stadt ===
land_city_map = {
    "Austria": "Wien",
    "Germany": "Berlin",
    "France": "Paris",
    "Spain": "Madrid",
    "Netherlands": "Amsterdam"
}

# === Dropdown ===
dropdown = widgets.Dropdown(
    options=[(f"{city} ({land})", (city, land)) for land, city in land_city_map.items()],
    description="Ort auswählen:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# === Visualisierung ===
def update_plot(selection):
    city, land = selection

    # Temperaturdaten filtern
    temp_city = weather_df[weather_df["City"] == city]
    temp_daily = temp_city.groupby(temp_city["Date"].dt.date)["Temperature"].mean()

    # Energiedaten filtern
    region_code = energy_df[energy_df["Land"] == land]["sistema"].unique()
    if len(region_code) == 0:
        print(f"⚠️ Kein passender Region-Code für {land}")
        return
    code = region_code[0]
    energy_region = energy_df[energy_df["sistema"] == code]
    energy_daily = energy_region.groupby(energy_region["datetime"].dt.date)["precio"].mean()

    # Kombinieren
    combined = pd.DataFrame({
        "Temperature": temp_daily,
        "EnergyPrice": energy_daily.reindex(temp_daily.index)
    }).dropna()

    if combined.empty:
        print(f"⚠️ Nicht genügend Daten für {city} ({land}).")
        return

    # Korrelation
    corr, p = pearsonr(combined["Temperature"], combined["EnergyPrice"])

    # Zeitverlauf
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    axs[0].plot(combined.index, combined["Temperature"], color="red")
    axs[0].set_title(f"Temperatur in {city} (2020)")
    axs[0].set_ylabel("Temperatur [°C]")
    axs[0].tick_params(axis='x', rotation=45)

    axs[1].plot(combined.index, combined["EnergyPrice"], color="blue")
    axs[1].set_title(f"Energiepreis in {land} (2020)")
    axs[1].set_ylabel("Preis [€/MWh]")
    axs[1].tick_params(axis='x', rotation=45)

    plt.suptitle(f"Korrelation: r = {corr:.2f}, p = {p:.1e}")
    plt.tight_layout()
    plt.show()

    # Scatterplot
    plt.figure(figsize=(6,5))
    plt.scatter(combined["Temperature"], combined["EnergyPrice"], alpha=0.6)
    plt.xlabel("Temperatur [°C]")
    plt.ylabel("Energiepreis [€/MWh]")
    plt.title(f"Scatterplot: {city} / {land}\nr = {corr:.2f}, p = {p:.1e}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Interpretation
    print("📊 Pearson-Korrelation:", f"{corr:.3f}")
    print("🧪 p-Wert:", f"{p:.2e}")
    print("💬 Interpretation:")
    if p < 0.05:
        print("✅ Statistisch signifikant.")
        if corr < 0:
            print("→ Höhere Temperaturen gehen mit niedrigeren Energiepreisen einher.")
        else:
            print("→ Höhere Temperaturen gehen mit höheren Energiepreisen einher.")
    else:
        print("⚠️ Kein signifikanter Zusammenhang feststellbar.")

# === Anzeige starten ===
widgets.interact(update_plot, selection=dropdown)


interactive(children=(Dropdown(description='Ort auswählen:', layout=Layout(width='50%'), options=(('Wien (Aust…

<function __main__.update_plot(selection)>